##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Getting started with Gemini models

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Get_started.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

The new **[Google Gen AI SDK](https://github.com/googleapis/python-genai)** provides a unified interface to [Gemini models](https://ai.google.dev/gemini-api/docs/models) through both the [Gemini Developer API](https://ai.google.dev/gemini-api/docs) and the Gemini API on [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview). With a few exceptions, code that runs on one platform will run on both. This notebook uses the Developer API.

This notebook will walk you through:

* [Installing and setting-up](Get_started.ipynb#scrollTo=Mfk6YY3G5kqp) the Google GenAI SDK
* [Text](Get_started.ipynb#scrollTo=6TYNPrNvQ8ue) and [multimodal](#scrollTo=yww-vrxmRiIy) prompting
* Counting [tokens](Get_started.ipynb#scrollTo=_9B8pb7tv_Cx)
* Setting system instructions
* Configuring [safety filters](Get_started.ipynb#scrollTo=HTAnYx_bbxPk)
* Initiating a [multi-turn chat](Get_started.ipynb#scrollTo=HTAnYx_bbxPk)
* [Controlling generated output](Get_started.ipynb#scrollTo=nyZMoM6tgnTA)
* Using [function calling](Get_started.ipynb#scrollTo=Rl-y9SZywD0s)
* Generating a [content stream](Get_started.ipynb#scrollTo=uQfLCxfQtPTg) and sending [asynchronous](Get_started.ipynb#scrollTo=plCtEIaHuv96) requests
* Using [file uploads](Get_started.ipynb#scrollTo=enBhuaIk3KYa)
* Using [context caching](Get_started.ipynb#scrollTo=oTgeR3_9wN5J)
* Generating [text embeddings](Get_started.ipynb#scrollTo=sXNCRn8Wx71d)

More details about this new SDK on the [documentation](https://ai.google.dev/gemini-api/docs/sdks).

## Setup

### Install SDK

Install the SDK from [PyPI](https://github.com/googleapis/python-genai).

In [2]:
%pip install -U -q 'google-genai>=1.0.0'

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key or you aren't sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [3]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

### Initialize SDK client

With the new SDK, now you only need to initialize a client with you API key (or OAuth if using [Vertex AI](https://cloud.google.com/vertex-ai)). The model is now set in each call.

In [4]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

### Choose a model

Select the model you want to use in this guide. You can either select one from the list or enter a model name manually. Keep in mind that some models, such as the 2.5 ones are thinking models and thus take slightly more time to respond. For more details, you can see [thinking notebook](./Get_started_thinking.ipynb) to learn how to switch the thinking off.

For a full overview of all Gemini models, check the [documentation](https://ai.google.dev/gemini-api/docs/models/gemini).

In [5]:
MODEL_ID = "gemini-2.5-flash" # @param ["gemini-2.5-flash-lite-preview-06-17", "gemini-2.5-flash", "gemini-2.5-pro"] {"allow-input":true, isTemplate: true}

## Send text prompts

Use the `generate_content` method to generate responses to your prompts. You can pass text directly to `generate_content` and use the `.text` property to get the text content of the response. Note that the `.text` field will work when there's only one part in the output.

In [6]:
from IPython.display import Markdown

prefix="""
You are a saadhak (god seeker) listening to Swami Mukundanda's lectures.

You missed a live lecture. However, a friend of yours gave you the script used by Swamiji.

However, you are not perfect and can't draw conclusions, so you take very careful steps.

Create a 150 word Hindi summary of this excerpt.
Re-read and edit multiple times so that meaning is unchanged. Skip the niceties and start with the summary.
"""

transcript = """
हम जो स्व संवाद करते हैं उस पर हम लोग ध्यान नहीं देते सेल्फ टॉक जैसे स्वामी जी का प्रवचन सुन रहे और मन ही मन बात करते चले जा रहे हैं अब बात तो ठीक कह रहा इस पॉइंट को उसने ज्यादा लंबा खींच दिया दाढ़ी वाला समझता क्या अपने आप को व निरंतर बात चलती चली जाती सुनते भी जाते हैं बात भी चलती जाती इतना सारा सेल्फ टॉक और वह जो सेल्फ टॉक अगर गलत है तो क्या परिणाम होगा हमारे ऊपर और उसी को हम सुधार द भगवान की बड़ी कृपा है भगवान रक्षा कर रहे हैं भगवान का मेरे लिए बहुत बढ़िया प्लान है जब आप इसको रिपीट करेंगे बारबार बार बार तो अवचेतन मन सकारात्मक बन जाएगा [संगीत] [प्रशंसा] किसी ने कहा कि हमारे अंदर दो चैनल चलते हैं एक चैनल पी एक चैनल ए जैसे रेडियो के अलग-अलग चैनल होते हैं ना तो ऐसे ही मन के दो चैनल है एक चैनल पी एक चैनल ए पी मतलब पॉजिटिविटी सकारात्मक एन मतलब नेगेटिविटी नकारात्मक जैसे मान लो एक मार्केटिंग का एजेंट है अब वह सामान बेचने गया और सामान दूसरे ग्राहक ने खरीदा नहीं तो चैनल एन कहता है अरे यह सामान ही बेकार है तुम बेच रहे हो तुमको तो बेचना आता ही नहीं है तुमको कभी सफलता नहीं मिलेगी ये चैनल एन नकारात्मक बोलता जाता है नेगेटिव विचार और चैनल भी वो कहता है चलो कोई बात नहीं यहां नहीं बिका अगले वाले को बेच देंगे सामान को और बढ़िया कर देंगे जो हमारे अंदर कमी है उसको सुधार लेंगे तो दोनों स्टेशन चल रहे हैं मन में अब हम जिसको सुनेंगे वैसे हमारे भाव बन जाएंगे तो दुर्भाग्य ये है कि हम जो स्व संवाद करते हैं उस पर हम लोग ध्यान नहीं देते सेल्फ टॉक दूसरों से बात करते लेकिन अपने से भी बात करते जैसे स्वामी जी का प्रवचन सुन रहे और मन ही मन बात करते चले जा रहे हैं अब बात तो ठीक कह रहा है इस पॉइंट को उसने ज्यादा लंबा खींच दिया दाढ़ी वाला समझता क्या अपने आप को वो निरंतर बात चलती चली जाती सुनते भी जाते हैं बात भी चलती जाती आप कहेंगे स्वामी जी हम अपने आप से बात करते हैं क्या बिल्कुल करते हैं देखिए लोग तो गोल्फ बॉल से बिलियर्ड बॉल से बात करते हैं व गोल्फ में बॉल होता है ना जा जा जा तीन इच रह गए तीन च रह गए बिलियन बॉल के साथ व बिलियन खेलते ना हमने कई बार लोगों को देखा बातचीत कर रहे हैं तो जब बॉल के साथ कर सकते हैं क्या अपने साथ नहीं करते बच्चे लोग जो होते हैं वह अपने साथ जोर जोर से बात करते हैं एक बार मैं किसी के घर में ठहरा था अमेरिका में और हल्ला मच गया मैंने सोचा भी पता नहीं क्या बवाल हो गया बाहर गया तो उनके बच्चे के कमरे से हल्ला आ रहा था तो वहां मैंने झाक के देखा बच्चा भालू डॉल इत्यादि के साथ खेल रहा था तो जोर जोर से बातें कर रहा था अब हम जब बड़े हो गए तो जो जोर से सेल्फ टॉक नहीं करते जोर जोर से करेंगे तो लोग कहेंगे पागल है जैसे आप कभी कभी देखते हैं ना सड़क पर आदमी ज्यारा जोर से बातें करते हुए अपने साथ आप कहते हैं पागल है तो इसलिए हम चुपचाप बातें करते चले जाते लेकिन वो जो हम स्व संवाद करते हैं वह सबसे इंपॉर्टेंट है आपने स्वामी जी से क्या सुना इसका महत्व नहीं है आप हज बार बार सुने और वैसे के वैसे रह जाए सुनने के बाद आपने अपने आप को क्या कहा इसका महत्व है अब हम अपने आप स्व संवाद को बदल दे बस प्रोग्रामिंग चेंज हो जाएगी एक मान्यता थी पिछली शताब्दी में जब ओलंपिक्स मॉडर्न ओलंपिक्स शुरू हुए 18960 वर्ष तक कोई एक मील की दूरी को 4 मिनट से कम समय में नहीं भाग पाया था यानी वह एक माइल को लेस देन फोर मिनट्स में कोई नहीं भाग पा रहा था और सब यही कहते थे कि ये तो संभव ही नहीं वैज्ञानिक लोग बताते थे भाई एक मील को चार मिनट से जल्दी भागने का प्रयत्न करोगे तो तुम्हारे फेफड़े जो हैं वह छेद से फट जाएंगे कोई कहता था तुम्हारा तो हार्ट अटैक हो जाएगा कोई कहता था कि भगवान ने ऐसी मांसपेशियां ही नहीं दी तो एक इंग्लैंड का खिलाड़ी था स्पोर्ट्स पर्सन उसका नाम था रॉजर बनिस्टर वो एथलेटिक्स में में पीएचडी कर रहा था और एक मील को भागने के लिए वह चार मिनट सा सेकंड लगाता चार मिनट से ऊपर ही था उसका समय लेकिन उसने सेल्फ टॉक को चेंज कर दिया उसने कहा मैं फोर मिनट बैरियर को तोड़ दूंगा लोगों ने कहा तुम कैसे तोड़ दोगे तुम तो फोर मिनट सेन सेकंड में भागते हो उसने कहा मैं तोड़ दूंगा अब वो टॉक करता था परिणाम स्वरूप 1906 में उसने फोर मिनट से कम समय में एक मील को भाग लिया और आश्चर्य की बात यह हुई कि जब उसने वह रिकॉर्ड को तोड़ा तो विश्व भर के एथलीट्स का सेल्फ टॉक स्व संवाद बदल गया सब सोचने लग गए यह तो हो सकता यह तो संभव है इसमें कोई असंभव बात नहीं है और जब वे ऐसा सोचने लगे अवचेतन मन सब की प्रो हो गई अब वह सब उसी वर्ष 23 रनर्स ने चार मिनट से कम समय में एक मील भागी और अगले वर्ष तो 275 लोग फोर मिनट बैरियर को तोड़ दिए तो यह अवचेतन मन हमें स्वस्थ कर सकता है अवचेतन मन हमें अस्वस्थ कर सकता है इतना शक्ति ली हमको वो अवचेतन मन को अपना दास बनाना है अपना सहायक फ्रेंड और उसके लिए हम जो स स्व संवाद करते हैं उसका सुधार करना इसको कहते हैं पॉजिटिव एफर्मेशन अच्छी बात को बार-बार लाओ अब कोई एक मिनट में मान लो ढ स विचार लाता है अपने मन एक मिनट में यह सोचे वो सोचे यह सोचे वो सोचे तो आप सोचिए दिन भर में हम लगभग 5 लाख विचार ले आए मन में इतना सारा सेल्फ टॉक और वह जो सेल्फ टॉक अगर गलत है तो क्या परिणाम होगा हमारे ऊपर और उसी को हम सुधार दे भगवान की बड़ी कृपा है भगवान रक्षा कर रहे हैं भगवान का मेरे लिए एक बहुत बढ़िया प्लान जब आप इसको रिपीट करेंगे बारबार बार बार तो अवचेतन मन सकारात्मक बन जाएगा तो यह स्व संवाद का महत्व और इसी का दूसरा स्वरूप हमारे भक्ति संतों ने बताया भक्ति संतों ने कहा कि जीवों को आत्म शुद्धि का हम साधन बताते यदि वे स्व संवाद को भगवन मय कर देंगे तो अवचेतन मन भगवन मय हो जाएगा कैसे भगवन मय करें उन्होंने कहा श्वास शवास से भगवान का नाम जपो श्वास शवास सुमिरन करो यह उपाय बड़ो नीक ये गुरु नानक कहते श्वासा माही जपते दुविधा रहे न कोई संत कबीर कहते और हमारे महाराज जी ने तो भर दिया उसमें कि भ राधे राधे ऐसे करो तो जब बारबार बार बार नाम जपोगे मन ही मन जब नाम को रिपीट करते जाओगे तो उससे तुम्हारा अवचेतन मन भगवन मय होने लग जाएगा [संगीत] [प्रशंसा] [संगीत] [संगीत] [संगीत] माइंड मैनेजमेंट लाइफ ट्रांसफॉर्मेशन पावर ऑफ थॉट्स आदि प्रेरणादायक एवं आध्यात्मिक वीडियोस के लिए सब्सक्राइब करें स्वामी मुकुंदा हिंदी और बेल आइकन को अवश्य दबाएं ताकि आपको यह वीडियोस मिले सबसे पहले [संगीत] [संगीत]
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prefix+transcript
)

Markdown(response.text)

स्वामी मुकुंदानंद बताते हैं कि हमारा स्व-संवाद (self-talk) अत्यंत महत्वपूर्ण है, जिस पर हम ध्यान नहीं देते। मन में सकारात्मक और नकारात्मक दो चैनल होते हैं; हम जिसे चुनते हैं, वही हमारे भाव बनाता है। बाहरी बातें सुनने से अधिक अहम है, हम खुद से क्या कहते हैं।

रॉजर बैनिस्टर ने अपनी आंतरिक बातचीत बदलकर चार मिनट मील दौड़ने का असंभव रिकॉर्ड तोड़ा, जिससे अवचेतन मन प्रभावित हुआ। हमारा अवचेतन मन शक्तिशाली है; इसे सहायक बनाने के लिए स्व-संवाद को सकारात्मक बनाना होगा। बार-बार अच्छी बातें दोहराने (positive affirmations) से यह संभव है।

भक्ति संतों ने स्व-संवाद को भगवनमय बनाने का उपाय बताया: श्वास-श्वास भगवान का नाम जपना। इससे अवचेतन मन दिव्य बनता है और जीवन में सकारात्मकता आती है। गलत स्व-संवाद नकारात्मक परिणाम देता है। अतः, अपनी आंतरिक बातचीत को सुधारना ही उन्नति का मार्ग है।